# FIT5196 Task 1 in Assessment 2
#### Student Name: Boyu Zhang
#### Student ID: 28491300

Date: XX/XX/XXXX

Version: 1.0

Environment: Python 3.6 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.,:
* pandas (for dataframe, included in Anaconda Python ) 
* re (for regular expression, included in Anaconda Python 3.6) 
* itertools(for creation of string pairs, included in Python 3.6)
* numpy (for numpy array, included in Anaconda Python 3.6) 
* jellyfish (for string similarity computation,various algorithms, obtained by `pip install jellyfish`)



## 1.  Import libraries 

In [1]:
# Code to import libraries as you need in this assessment, e.g.,
import pandas as pd
import numpy as np
import re
from itertools import combinations
import jellyfish

## 2. Load data and exploration


In [2]:
df1 = pd.read_csv('./Input/dataset1_with_error.csv')

In [3]:
#overview of the dataset
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25077 entries, 0 to 25076
Data columns (total 11 columns):
Id                  25077 non-null int64
Title               25077 non-null object
Location            25077 non-null object
ContractType        25077 non-null object
ContractTime        25077 non-null object
Company             21242 non-null object
Category            25077 non-null object
Salary per annum    25077 non-null object
SourceName          25077 non-null object
OpenDate            25077 non-null object
CloseDate           25077 non-null object
dtypes: int64(1), object(10)
memory usage: 2.1+ MB


In [4]:
df1.describe(include=['O'])

,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
count,25077,25077,25077,25077,21242,25077,25077,25077,25077,25077
unique,25077,482,3,3,4879,8,1589,90,2203,2400
top,"Carpentry Trainer / Technician (Assessor, Voca...",UK,not available,permanent,UKStaffsearch,IT Jobs,35004,totaljobs.com,20130127T150000,20130526T120000
freq,1,3996,19499,16194,248,7085,1011,5335,27,23


In [5]:
#number of missing values in each column
df1.isnull().sum()

Id                     0
Title                  0
Location               0
ContractType           0
ContractTime           0
Company             3835
Category               0
Salary per annum       0
SourceName             0
OpenDate               0
CloseDate              0
dtype: int64

### Highlight of initial investigation
- There are 9 columns of categorical values, and 2 column of numeric values -- `Id` and `Salary`
- There are a lot of missing values in the `Company` column
- There must be issue with the column `Title` since the most frequent value is like "** *Ton**"which is obviously invalid
- The most frequent value of `Location` is "UK" which suspicious
- There are potential semantic anomalies :
    - unique values can have redundance
    - The violation of two Date
- There can be sytax anomalies:
     - inconistent representation of values in the same column
     - invalid value of categorical values

In [6]:
#first glance of the dataset
df1.head()

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
0,12612628,Engineering Systems Analyst,Dorking,not available,permanent,Gregory Martin International,Engineering Jobs,24996,cv-library.co.uk,20121103T000000,20121203T000000
1,12612830,Stress Engineer Glasgow,Glasgow,not available,permanent,Gregory Martin International,Engineering Jobs,30000,cv-library.co.uk,20130108T150000,20130408T150000
2,12612844,Modelling and simulation analyst,Hampshire,not available,permanent,Gregory Martin International,Engineering Jobs,30000,cv-library.co.uk,20130726T150000,20130924T150000
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Surrey,not available,permanent,Gregory Martin International,Engineering Jobs,27504,cv-library.co.uk,20121214T000000,20130314T000000
4,12613647,"Pioneer, Miser Engineering Systems Analyst",Surrey,not available,permanent,Gregory Martin International,Engineering Jobs,24996,cv-library.co.uk,20131025T000000,20131224T000000


## 3. Identify issues and fix 
- If there is clear and feasible fix for the error, there will be explicit section stating the **Fix** method and content;

- Otherwise, we only provide a **Conclusion** for the issue

### ID
For the column `ID` we don't have any constraints other than non-redundance. Thus we only check whether all **IDs** is unique.

In [7]:
#test if all Id 
df1.Id.unique().shape[0] == df1.shape[0]

True

### Title
- It's reasonable whether there are redundant tiltles or not.
- The values should be in valid language

In [8]:
#test whethere there is redundant titles
df1.Title.unique().shape

(25077,)

In [9]:
df1.Title.value_counts()

Carpentry Trainer / Technician (Assessor, Vocational, Training) BLUT****     1
Nurse Advisor                                                                1
Conference and Banqueting Manager                                            1
Systems Admin / Dev Ops Engineer                                             1
KS2 Teacher Year ****/**** Class                                             1
C NET Microsoft Developer                                                    1
Sage 200 Project Manager/BI/Financials Cambridgeshire ****k                  1
Financial Accountant  Market Leading Insurer                                 1
Staff Nurse  HenleyonThames                                                  1
Registered Care Manager Sheffield                                            1
International Marketing Executive                                            1
Care Assistant  Bournemouth                                                  1
IT Service Delivery Manager (SDM)                   

- It appears there are values that contain invalid characters '*' in the `Title` column 
- It seems some titles also contains the company name or location which could be improved for clearity of data

In [10]:
df1[df1.Title=='Engine CMPT Leader Cranktrain System  POW****']

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
13717,68544614,Engine CMPT Leader Cranktrain System POW****,Gaydon,not available,not available,Matchtech Group plc.,Engineering Jobs,52800,rengineeringjobs.com,20130516T000000,20130615T000000


In [11]:
df1[df1.Title.str.contains('Year')]

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
904,48271471,Early Years Lead Practitioner (Children s Centre),Manchester,not available,not available,NaN,Healthcare & Nursing Jobs,43200,careworx.co.uk,20120715T120000,20120814T120000
2224,62114019,Year Six Teacher,UK,full_time,permanent,Edustaff,Teaching Jobs,36504,fish4.co.uk,20120531T120000,20120829T120000
2235,62114222,Year **** Primary School Teacher,Birmingham,full_time,permanent,Edustaff Birmingham,Teaching Jobs,30792,fish4.co.uk,20130924T120000,20131123T120000
2257,62115888,Year 5 Teacher,Birmingham,full_time,permanent,Balfor Education,Teaching Jobs,28800,fish4.co.uk,20120731T120000,20121029T120000
2288,62119102,Year 1 Teacher at London Bridge Up to **** pe...,UK,full_time,permanent,Destination Education,Teaching Jobs,34800,fish4.co.uk,20120805T000000,20121004T000000
2289,62119106,Primary School Teacher Year **** Class,Coventry,full_time,permanent,Edustaff Birmingham,Teaching Jobs,30792,fish4.co.uk,20130217T000000,20130319T000000
2342,62124569,Year ****,UK,full_time,permanent,Career Teachers,Teaching Jobs,31200,fish4.co.uk,20131118T120000,20131218T120000
2625,64214875,Year 3 Teacher,Enfield,full_time,permanent,Edustaff,Teaching Jobs,33060,fish4.co.uk,20120511T150000,20120809T150000
2637,64339591,Year **** (Literacy Numeracy Support) Teacher ...,London,full_time,contract,Tradewind Recruitment,Teaching Jobs,31800,jobsineducation.co.uk,20130219T150000,20130420T150000
2866,64802114,Year 7 Teacher,London,full_time,permanent,Harris Hill,Teaching Jobs,32496,jobsineducation.co.uk,20130910T120000,20131109T120000


In [12]:
df1[df1.Title=='PHP Developer, LAMP Software Engineer ****k']

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
9960,68180029,"PHP Developer, LAMP Software Engineer ****k",London,not available,permanent,Acceler IT Recruitment Consultancy,IT Jobs,65004,jobsite.co.uk,20120304T120000,20120602T120000


#### Fix:

- We can unify the capital letters in the title column

In [13]:
#unify string
df1.Title = df1.Title.str.title()

#### Conclusions:
- After inspectation we find out that there are a considerable number of astericks, however, quite a number of them has valid semantic meaning. It is just these infomartion somehow missed, like "Year \*\** Class Teach" or "Engineer \**k" to these entities the info represented by the astericks are crucial for it to be understood, these astericks can at least provide clues on  that. Thus we decide to keep these astericks.

**Keep unchanged**

### Location 


In [14]:
df1.Location.value_counts()

UK                      3996
London                  2743
South East London       1433
The City                 558
Leeds                    343
Manchester               334
Surrey                   317
Central London           316
Reading                  300
Birmingham               292
West Midlands            273
East Sheen               222
Bristol                  222
Berkshire                202
Nottingham               196
Oxford                   190
Hampshire                184
Milton Keynes            180
Cambridge                172
Sheffield                160
Kent                     158
Newcastle Upon Tyne      156
Hertfordshire            155
Guildford                147
Aberdeen                 141
Leicester                139
Cheshire                 133
West Yorkshire           127
Oxfordshire              127
Belfast                  125
                        ... 
Twickenham                 4
North Finchley             4
Wisbech                    4
Bexley        

Potential issues:
1. A large proportion records has just no detailed `Location` but just "UK"(default filling for none);some has the location data but it is included in the `Title` value and these record also just have the value of `Location` as "UK"
2. the "The City" location does not make sense.
3. There are several occurence of "London" like "London", "Central London","South East London".
4. There seems to be spelling errors like "Oxfords" and "Oxford"

So we start to check these potention issues one by one:

In [15]:
df1[df1.Location == 'UK']

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
9,22933091,Chef De Partie Award Winning Dining Live In ...,UK,not available,not available,Chef Results,Hospitality & Catering Jobs,18000,caterer.com,20120814T000000,20120828T000000
12,23530231,Lead Engineers (Stress),UK,not available,permanent,Gregory Martin International,Engineering Jobs,39504,cv-library.co.uk,20121113T120000,20130211T120000
13,24104334,"Relief Chef De Partie Croydon, Surrey Live In",UK,not available,not available,Towngate Personnel,Hospitality & Catering Jobs,17280,caterer.com,20131029T000000,20131128T000000
26,27739522,Welwyn Chef De Partie Does It Get Any Better T...,UK,not available,not available,Clear Selection,Hospitality & Catering Jobs,18000,caterer.com,20130106T120000,20130307T120000
27,27740574,Pastry Chef Al**** ****Aa Rosette Restaurant,UK,not available,not available,Clear Selection,Hospitality & Catering Jobs,18996,caterer.com,20120320T120000,20120403T120000
30,29650985,Fine Dining Chefs For Rosette Restaurants Up ...,UK,not available,not available,Clear Selection,Hospitality & Catering Jobs,24000,caterer.com,20131205T150000,20140104T150000
31,30192171,Web Developer,UK,not available,permanent,Indigo 21 Ltd,IT Jobs,35004,cv-library.co.uk,20130305T000000,20130504T000000
39,31989124,Mechanical Engineer Design And Substantiation,UK,not available,permanent,Gregory Martin International,Engineering Jobs,35004,cv-library.co.uk,20131018T120000,20131117T120000
42,32233421,Principle Mechanical Engineer,UK,not available,permanent,Rullion Engineering Ltd,Engineering Jobs,48504,cv-library.co.uk,20130816T120000,20131114T120000
43,32417995,Chef De Partie Award Winning Fine Dining Rest...,UK,not available,not available,Chef Results,Hospitality & Catering Jobs,20496,caterer.com,20131016T150000,20131030T150000


In [16]:
df1[df1.Location == 'The City']

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
51,34479775,Credit Controller,The City,not available,permanent,NaN,Accounting & Finance Jobs,24000,hays.co.uk,20120326T000000,20120425T000000
692,46635557,Occupational Health Advisor London,The City,full_time,not available,NaN,Healthcare & Nursing Jobs,38004,careworx.co.uk,20130325T150000,20130424T150000
697,46635605,Practice Nurse,The City,not available,not available,NaN,Healthcare & Nursing Jobs,34500,careworx.co.uk,20120704T120000,20120803T120000
711,46635738,Senior Staff Nurse Itu,The City,not available,not available,NaN,Healthcare & Nursing Jobs,33000,careworx.co.uk,20130105T150000,20130204T150000
761,46636393,"Deputy Manager, Elderly Nursing Home, Chingford",The City,not available,not available,NaN,Healthcare & Nursing Jobs,33996,careworx.co.uk,20120615T150000,20120629T150000
763,46636397,"Lead Nurse, Edmonton, Elderly Nursing Home",The City,not available,not available,NaN,Healthcare & Nursing Jobs,27000,careworx.co.uk,20120130T120000,20120229T120000
764,46636398,"Unit Manager, Edmonton, Elderly Care Nursing Home",The City,not available,not available,NaN,Healthcare & Nursing Jobs,26004,careworx.co.uk,20120526T000000,20120725T000000
846,46852053,Clinical Operations Manager,The City,not available,not available,NaN,Healthcare & Nursing Jobs,42000,careworx.co.uk,20120112T000000,20120211T000000
932,48481518,Team Health Administrator,The City,not available,not available,NaN,Healthcare & Nursing Jobs,24996,careworx.co.uk,20120626T120000,20120710T120000
982,49689034,Governance Manager,The City,not available,not available,NaN,Healthcare & Nursing Jobs,37404,careworx.co.uk,20130803T000000,20131002T000000


In [17]:
#find similarity of strings
def find_close_strings(target):
    word_list = list(df1.Location.unique())
    pairs = []
    for e in combinations(word_list,2):
        pairs.append(e)
    result = []
    for pair in pairs:
        if jellyfish.damerau_levenshtein_distance(pair[0],pair[1]) <= target:
            result.append(pair)
    return result

After several tests, we decide to pick the **damerau_levenshtein** algorithm to quantify string difference, and we assume that if the distance between two strings is 1, then there is high possibility they are the same word

In [18]:
find_close_strings(1)

[('Surrey', 'Surey'),
 ('Hertfordshire', 'Herefordshire'),
 ('Leeds', 'Leads'),
 ('Nottingham', 'Nottinham'),
 ('Boston', 'Bolton'),
 ('Harlow', 'Harrow'),
 ('Harlow', 'Marlow'),
 ('Oxford', 'Oxfords'),
 ('Hereford', 'Hertford'),
 ('Reeding', 'Reading'),
 ('Fareham', 'Farnham'),
 ('Stafford', 'Stamford'),
 ('Tamworth', 'Tadworth'),
 ('Uxbridge', 'Axbridge')]

#### Fix:
After search on the internet we refine the list of string-pairs that should be rectified

In [19]:
#fix the lexical error in location name
to_correct = {'Surey':'Surrey','Leads':'Leeds','Oxfords':'Oxford','Reeding':'Reading','Nottinham':'Nottingham'}
df1.replace({'Location':to_correct},inplace=True)

#### Conclusion:
1. The "UK" seems to be the default value of the `Location` column, though there are some records which has the `Location` in the `Title`, we cannot extract it since we have no way to know if it is a valid location name or not.**Cannot clean**
2. Among records with "The City" as Location info, some has Location info in title, some has not, same as above, no way to tell if it is a location or not.**Cannot clean**
3. Since London is a metropolitan, it is reasonable to keep location sepcific to which area of it, so we keep the mulitiple ocurrences as they are.

### ContractType

In [20]:
#check unique values of the columns
df1.ContractType.unique()

array(['not available', 'full_time', 'part_time'], dtype=object)

**Fix**:

According to dta dataset description, the 'not available' should be replaced by 'non_specified'

In [21]:
df1.ContractType.replace('not available','non_specified',inplace=True)

### ContractTime

In [22]:
#check unique values
df1.ContractTime.unique()

array(['permanent', 'not available', 'contract'], dtype=object)

**Fix**:

According to dta dataset description, the 'not available' should be replaced by 'non_specified'

In [23]:
df1.ContractTime.replace('not available','non_specified',inplace=True)

### Company

In [24]:
df1.Company.value_counts()

UKStaffsearch                        248
CVbrowser                            170
Randstad                             169
JOBG8                                159
Matchtech Group plc.                 150
Chef Results                         137
Penguin Recruitment                  134
Hays                                 115
London4Jobs                          112
COREcruitment International          111
JAM Recruitment Ltd                  109
Computer People                       99
Clear Selection                       92
Monarch Recruitment                   89
Towngate Personnel                    79
Cherryred Recruitment                 69
Senitor Associates                    67
Capita Resourcing                     62
Modis                                 62
Support Services Group                62
Aspire Data Recruitment               61
Adecco                                57
Idex Consulting                       55
Evolution Recruitment Solutions       55
Populus Healthca

In [25]:
df1.Company.str.contains('LTD').sum(),df1.Company.str.contains('Limited').sum(),df1.Company.str.contains('Ltd').sum()

(153, 1251, 3604)

- Seems there are some minor irregularities like the inconsitent use of "Limited","Ltd" and "LTD", which can all be unified as "Ltd"
- To be consistent "plc" can be converted into "Plc"

It is already discovered before that there is large amount of missing values in the `Company` column

In [26]:
# unify all post-fix of company name
df1.Company = df1.Company.str.title()
df1.Company = df1.Company.str.replace('Limited','Ltd')

In [27]:
df1.Company.str.contains('Limited').sum()

0

In [28]:
df1.Company.isna().sum()

3835

#### Conclusion:
It seems that it is impossible to fill in the name of these companies, but depends on whether it was just a system error or typo, and also regarding different usage of the dataset, it is better we keep this records.

### Category

In [29]:
df1.Category.value_counts()

IT Jobs                             7085
Healthcare & Nursing Jobs           4334
Engineering Jobs                    3458
Accounting & Finance Jobs           3099
Sales Jobs                          2609
Hospitality & Catering Jobs         2124
Teaching Jobs                       1378
PR, Advertising & Marketing Jobs     990
Name: Category, dtype: int64

#### Conclusion:
- No invalid values and no redundance
- Also no inconsistent represnetation of same value

### Salary

In [30]:
df1["Salary per annum"].unique()

array(['24996', '30000', '27504', ..., '17K', '18264', '40368'], dtype=object)

There are inconsiten representations of the salary

In [31]:
p1 = '^\d+$'

In [32]:
non_numeric_salary = df1['Salary per annum'].apply(lambda x: False if re.match(p1,x) else True )

In [33]:
inconsis_salary =df1['Salary per annum'][non_numeric_salary];inconsis_salary.value_counts()

32K                                        7
37996.2 - 41995.8                          5
33253.799999999996 - 36754.200000000004    5
35K                                        5
45K                                        5
30871.199999999997 - 34120.8               5
38K                                        5
40K                                        4
23K                                        4
26K                                        4
20K                                        4
30K                                        4
19K                                        4
28500.0 - 31500.0                          4
42750.0 - 47250.0                          4
24K                                        4
47K                                        4
52K                                        3
15196.199999999999 - 16795.8               3
27553.8 - 30454.2                          3
28K                                        3
23746.199999999997 - 26245.800000000003    3
21K       

#### Fix:
There seems to have two type of invalid representations:
1. "90K"
2. "21819.6 - 24116.4"

**For the first one we just transfer the str representation to int.**

**For the second one we pick the mean of the range and round them to the nearest integer.**

In [34]:
#design the regex patterns
p1 = '(\d+)K'#regex for '90K'
p2 = '(\d+\.\d+)\s*-\s*(\d+\.\d+)'#regex for '3333.222 - 34444.333'
p3 = '^\d+$' # regex for normal representation

In [35]:
#the method for unifying the values of salaries
def unify_salary(sp):
    r1 = re.match(p1,sp)
    r2 = re.match(p2,sp)
    if r1:
        return r1.group(1) + '000'
    if r2:
        return float(r2.group(1))+float(r2.group(2))/2
    if re.match(p3,sp):
        return sp

In [36]:
df1['Salary per annum'] = df1['Salary per annum'].apply(unify_salary)

In [37]:
#for space saving, salary won't be that large, so transfer to int32 is enough
df1['Salary per annum'] = df1['Salary per annum'].astype(int);

Then we need to check if there are semantic errors in the salary column:
- one error can be the occurence of error or negative number

In [38]:
df1.shape[0] - (df1['Salary per annum'] <= 0).sum() == df1.shape[0]

True

Thia proves there is not zero or negative values in the row of `Salary per annum`

### SourceName

In [39]:
df1.SourceName.value_counts()

totaljobs.com                     5335
cv-library.co.uk                  4207
careworx.co.uk                    1777
cwjobs.co.uk                      1202
staffnurse.com                    1182
jobsite.co.uk                     1110
caterer.com                       1047
fish4.co.uk                        757
theitjobboard.co.uk                755
jobserve.com                       717
jobs.catererandhotelkeeper.com     640
planetrecruit.com                  460
rengineeringjobs.com               363
salestarget.co.uk                  355
britishjobsonthe.net               332
jobs.guardian.co.uk                301
ukstaffsearch.com                  248
hays.co.uk                         247
strike-jobs.co.uk                  221
technojobs.co.uk                   219
gojobsearch.co.uk                  187
justengineers.net                  179
cvbrowser.com                      170
cityjobs.com                       159
jobg8.com                          145
careerbuilder.com        

Seems no explicit issues.

### OpenData and CloseDate
We assume at the first place that the date form is Y-M-D

In [40]:
df1.OpenDate.head()

0    20121103T000000
1    20130108T150000
2    20130726T150000
3    20121214T000000
4    20131025T000000
Name: OpenDate, dtype: object

The result of above cell proves our assumption, so we can start invetigate whether there are issues in the two columns for date.

In [41]:
p = '\d{4}(\d{2})(\d{2}).(\d{2})(\d{2})(\d{2})'
def is_valid_date(d):
    month_invalid = int(re.match(p,d).group(1)) > 12 
    day_invalid = int(re.match(p,d).group(2)) > 31 #keep it simple fallacious for the moment, for simplicity just assume each month has 31 days
    hr_invalid = int(re.match(p,d).group(3)) > 23
    min_invalid = int(re.match(p,d).group(4)) > 59
    second_invalid = int(re.match(p,d).group(5)) > 59
    if month_invalid or day_invalid or hr_invalid or min_invalid or second_invalid:
        return d
    else:
        return np.nan
    
invalid_open_date = df1.OpenDate.apply(is_valid_date)
invalid_close_date = df1.CloseDate.apply(is_valid_date)

In [42]:
#test
print(invalid_open_date.dropna().shape, invalid_close_date.dropna().shape)

(10,) (0,)


This indicates there are 10 invalid date 

Then we inspect the invalid values

In [43]:
invalid_series = invalid_open_date.dropna().values

In [44]:
df1[df1.OpenDate.isin(invalid_series)]

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
1102,51593546,Community Nurse South London,The City,non_specified,non_specified,NaN,Healthcare & Nursing Jobs,29004,careworx.co.uk,20131803T120000,20130417T120000
2104,61282006,Deputy Home Care Manager,Barnet,non_specified,non_specified,NaN,Healthcare & Nursing Jobs,20004,careworx.co.uk,20132606T000000,20130710T000000
2839,64796836,"Technical Tester Manual, Automation, Java, Sq...",London,non_specified,permanent,NaN,IT Jobs,19500,theitjobboard.co.uk,20122003T150000,20120519T150000
5707,66935867,"Technical Sales Manager, Pigments Fillers To S...",UK,non_specified,non_specified,Wallace Hind Selection,Sales Jobs,45000,salestarget.co.uk,20121512T150000,20130213T150000
10881,68257270,Desire A Change? London ****K Basic ****K Ote,South East London,non_specified,non_specified,Zest 2 Recruitment,Sales Jobs,13500,salestarget.co.uk,20133004T150000,20130530T150000
11948,68350622,Mechanical Design Engineer (Hvac),Gainsborough,non_specified,permanent,NaN,IT Jobs,39996,technojobs.co.uk,20131908T000000,20131117T000000
15353,68671948,Investment Analyst,Cheshire,non_specified,permanent,Idex Consulting,Accounting & Finance Jobs,24000,totaljobs.com,20121406T120000,20120912T120000
22918,69072305,Scheme Actuary,Hampshire,non_specified,permanent,Sammons Pensions,Accounting & Finance Jobs,54996,professionalpensionsjobs.com,20131509T000000,20131114T000000
23007,69079402,Naval Architect/Structural Engineer,Newcastle Upon Tyne,non_specified,permanent,Workshop Recruitment,Engineering Jobs,33504,cv-library.co.uk,20132901T150000,20130429T150000
23169,69080910,Ms (Elec Biased) Maintenance Engineer / Superv...,Harrogate,non_specified,permanent,Jenrick Engineering,Engineering Jobs,17004,cv-library.co.uk,20132108T120000,20131119T120000


#### Fix:
It seems though these OpenDates are invalid, but if the month and day get reversed they are all valid date.

Thus we decide to reverse them.

In [45]:
df1.loc[df1.OpenDate.isin(invalid_series), 'OpenDate'] = df1[df1.OpenDate.isin(invalid_series)]['OpenDate'].apply(lambda s: s[:4] + s[6:8] + s[4:6] + s[8:])

In [46]:
#transform the string representation of date to type datetime for more flexibility
df1[['OpenDate','CloseDate']] = df1[['OpenDate','CloseDate']].apply(lambda x: pd.to_datetime(x))

Then we need to check semantic errors, which is that that the OpenDate later than the CloseDate

In [47]:
df1[df1.OpenDate > df1.CloseDate]

,Id,Title,Location,ContractType,ContractTime,Company,Category,Salary per annum,SourceName,OpenDate,CloseDate
6659,67330693,Pensions Implementation Manager,Brighton,non_specified,permanent,NaN,Accounting & Finance Jobs,35004,hays.co.uk,2013-07-03 00:00:00,2013-04-04 00:00:00
9568,68091856,Manager Technical Operations,Shepton Mallet,non_specified,permanent,Index Recruitment,IT Jobs,62004,totaljobs.com,2012-03-06 00:00:00,2012-02-05 00:00:00
11043,68290941,Planning Enforcement Officer,London,full_time,contract,Randstad,Engineering Jobs,39360,jobs.planningresource.co.uk,2013-11-05 00:00:00,2013-10-06 00:00:00
12473,68360715,Penetration Tester / Reverse Engineer,UK,non_specified,contract,Vertex Solutions,IT Jobs,76908,cwjobs.co.uk,2013-12-22 15:00:00,2013-10-23 15:00:00
19142,68729089,Registered Home Manager Loughborough,Loughborough,full_time,non_specified,Compass Associates Ltd,Healthcare & Nursing Jobs,42504,staffnurse.com,2012-08-03 15:00:00,2012-05-05 15:00:00
24206,69173205,"User Interface/Ui Developer Javascript, Jquer...",London,non_specified,permanent,Dawson & Walsh,IT Jobs,45000,totaljobs.com,2013-06-22 15:00:00,2013-03-24 15:00:00
24297,69181738,Registered General Nurse (Rgn)/ Registered Men...,Poole,full_time,non_specified,Executive Care Group Ltd,Healthcare & Nursing Jobs,24960,staffnurse.com,2013-01-16 00:00:00,2012-10-18 00:00:00
25039,69228213,"Scientific (Physics, Or Chemistry, Or Electron...",Warrington,full_time,permanent,NaN,Engineering Jobs,24996,fish4.co.uk,2013-07-08 12:00:00,2013-06-08 12:00:00


#### Fix:
We propose a solution to this semantic error which is reverse the `OpenDate` and the `CloseDate`, which is reverse the `CloseDate` and `OpenDate`

In [48]:
df1_date = df1[['OpenDate','CloseDate']]

In [49]:
df1[['OpenDate','CloseDate']] = pd.concat([df1_date.min(axis=1), df1_date.max(axis=1)], axis=1)

In [50]:
#output the dataframe
df1.to_csv('./dataset1_solution.csv',index=False)

## 3. Summary
- There are quite a lot of ways to indentify dirty data, it is quite dependent on the method.
- It can happen from time to time that we cannot fix or need to preserve the seemingly unclean data
### Reference
 1. Jellyfish. Retrieved from: https://jellyfish.readthedocs.io/en/latest/